In [ ]:
%matplotlib notebook
import rospy
import actionlib
import actionlib.msg
import assignment_2_2022.msg
import jupyros
import time
import sys
import select
import os
import numpy as np
import matplotlib.pyplot as plt
from std_msgs.msg import String, Int32
from geometry_msgs.msg import PoseStamped
from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry

In [ ]:
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal, MoveBaseResult
from actionlib_msgs.msg import GoalStatus
from sensor_msgs.msg  import LaserScan
from rt1_second_assignment.msg import odom_custom_msg
import assignment_2_2022.msg
from assignment_2_2022.msg import My_pos_vel, PlanningActionGoal, PlanningAction, PlanningGoal
from assignment_2_2022.srv import Goals_number, Goals_numberRequest, Goals_numberResponse

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import display
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from matplotlib.widgets import Button
from matplotlib.animation import FuncAnimation
import threading
from std_srvs.srv import *

In [ ]:
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
import ipywidgets as widgets

In [ ]:
# Initialize the client node
rospy.init_node('ros_jupyter_interface')

# Counters
goal_reached = 0
obstacle_distance = 0
targets_cancelled = 0

# Global variables for coordinates
x_coord = []
y_coord = []

In [ ]:
def laser_callback(laser_scan):
    # Find closest obstacle
    min_range = min(laser_scan.ranges)
    if min_range == float('inf'):
        obstacle_distance = 'Infinity'
    else:
        obstacle_distance = '{:.1f}'.format(min_range)
    # Update GUI
    obstacle_distance_text.value = obstacle_distance

# Update the robot position plot and the text boxes
def update_plot(frame):
    ln.set_data(x_coord, y_coord)
    global target_counts
    target_counts = [goal_reached, targets_cancelled]
    for rect, count in zip(bar_plot.patches, target_counts):
        rect.set_height(count)
    fig.canvas.draw()
    return ln,

def position_callback(msg):
    global x_coord, y_coord
    # Retrieve the actual position
    x = msg.x
    y = msg.y
    robot_position_text.value = '({:.2f}, {:.2f})'.format(x, y)
    x_coord.append(x)
    y_coord.append(y)

def callback_result(msg):
    status = msg.status.status
    # If status = 3, the goal is reached
    if status == 3:
        increase_reached_targets()
    # If status = 2, the goal is cancelled
    elif status == 2:
        increase_not_reached_targets()

#Callback function for the subscriber to the /reached_targets topic
def increase_reached_targets():
    global target_counts
    global goal_reached
    reached_targets += 1

#Callback function for the subscriber to the /not_reached_targets topic
def increase_not_reached_targets():
    global target_counts
    global targets_cancelled
    targets_cancelled += 1
    
def goal_callback(msg):
    # Process the received goal message
    x = msg.client_goal.target_pose.pose.position.x
    y = msg.client_goal.target_pose.pose.position.y
    # Plot the target
    ax1.plot(x, y, 'bo') 

In [ ]:
# Create the figure and axis
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10))

# Create the line object for the robot position plot
ln, = ax1.plot([], [], 'ro')
# Set the plot limits
ax1.set_xlim(-10, 10)
ax1.set_ylim(-10, 10)

# Create the bar plot for the number of reached/not-reached targets
targets = ['Reached', 'Not Reached']
target_counts = [goal_reached, targets_cancelled]
bar_plot = ax2.bar(targets, target_counts)
ax2.set_ylim(0,5)

# Create GUI widgets
robot_position_text = widgets.Text(description='Robot position:', value='(0, 0)')
obstacle_distance_text = widgets.Text(description='Closest obstacle:', value='Infinity')

In [ ]:
rospy.Subscriber('/scan', LaserScan, laser_callback)
rospy.Subscriber('/reaching_goal/result', assignment_2_2022.msg.PlanningActionResult , callback_result)
rospy.Subscriber('/robot_informations', odom_custom_msg  , position_callback)
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)

In [ ]:
# Graphic Interface
gui_layout = widgets.VBox([robot_position_text, obstacle_distance_text])
display(gui_layout)
ani = FuncAnimation(fig, update_plot, interval=100)
plt.show()

In [ ]:
# Define callback function for target set button
def set_button_callback(sender):
    global x_goal, y_goal
    
    # Get x and y coordinates from text boxes
    x_goal = float(x_textbox.value)
    y_goal = float(y_textbox.value)

    #Creating the goal
    client_goal = PoseStamped()
    client_goal.pose.position.x = x_goal
    client_goal.pose.position.y = y_goal

    client_goal = assignment_2_2022.msg.PlanningGoal(client_goal)

    client.send_goal(client_goal)

In [ ]:
# Define callback function for target set button
def stop_button_callback(bot):
    global client
    client.wait_for_server()
    client.cancel_goal()

In [ ]:
client_goal = PlanningGoal()

In [ ]:
# Buttons definition: set and stop buttons
x_textbox = widgets.FloatText(description = 'X:')
y_textbox = widgets.FloatText(description = 'Y:')
set_button = widgets.Button(description = 'Set Target')
stop_button = widgets.Button(description = 'Cancel Target')
set_button.on_click(set_button_callback)
stop_button.on_click(stop_button_callback)
display(widgets.HBox([x_textbox, y_textbox, set_button, stop_button]))